In [ ]:
import cv2
import mediapipe as mp

# Initialize Mediapipe pose detector.
mp_pose = mp.solutions.pose
pose_detector = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
# Optional: for drawing, Mediapipe provides utilities.
mp_drawing = mp.solutions.drawing_utils

# Define custom skeleton edges (using Mediapipe's 33-point landmarks).
EDGES = [
    (0,1),(1,2),(2,3),(3,7),(0,4),(4,5),(5,6),(6,8),
    (9,10),(11,12),(12,14),(14,16),(11,13),(13,15),
    (12,24),(24,23),(23,11),(24,26),(26,28),(28,32),
    (23,25),(25,27),(27,31)
]

def overlay_skeleton(video_path, output_path=None):
    cap = cv2.VideoCapture(video_path)
    
    # Get video properties.
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    
    # Prepare video writer if output path is provided.
    if output_path is not None:
        fourcc = cv2.VideoWriter_fourcc(*'XVID') # 'XVID' is for .avi files.
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert frame to RGB for processing.
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose_detector.process(image_rgb)
        
        if results.pose_landmarks:
            # Option 1: Use Mediapipe's drawing utilities:
            # mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            # Option 2: Draw custom edges using our EDGES list.
            for (start_idx, end_idx) in EDGES:
                start = results.pose_landmarks.landmark[start_idx]
                end = results.pose_landmarks.landmark[end_idx]
                # Convert normalized coordinates to pixel values.
                start_point = (int(start.x * width), int(start.y * height))
                end_point   = (int(end.x * width), int(end.y * height))
                cv2.line(frame, start_point, end_point, (0, 255, 0), 2)
            
            # Draw joints as small circles.
            for lm in results.pose_landmarks.landmark:
                x, y = int(lm.x * width), int(lm.y * height)
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)
        
        # Display the frame.
        cv2.imshow("Skeleton Overlay", frame)
        if output_path is not None:
            out.write(frame)
        
        # Press 'q' to exit.
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    if output_path is not None:
        out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_file = "path/to/your/video.avi"  # Replace with your video path.
    output_file = "path/to/output_video.avi"  # Optional: output file path.
    overlay_skeleton(video_file, output_file)


In [4]:
video_file = r"C:\Users\johna\Desktop\CS3264proj\testing\vid\video (1).avi"  # Replace with your video path.
output_file = r"testing/output_video.avi"  # Optional: output file path.
overlay_skeleton(video_file, output_file)